In [ ]:
%%bash
pip install -r ../../requirements.txt
pip install mlflow


### Objective
Standardize splits, metrics, and experiment logging with MLflow.


In [ ]:
%%bash
set -e
python - <<'PY'
import json
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../../data/teacher_course.csv')
strat = df['sentiment'].astype(str) + '__' + df['aspect'].astype(str)
train_idx, val_idx = train_test_split(df.index, test_size=0.2, stratify=strat, random_state=42)
Path('../../outputs/splits').mkdir(parents=True, exist_ok=True)
Path('../../outputs/splits/default.json').write_text(json.dumps({'train': train_idx.tolist(), 'val': val_idx.tolist()}, indent=2))
PY


In [ ]:
%%bash
set -e
mlflow run ../.. -e sentiment_experiment --env-manager=local -P data=../../data/teacher_course.csv -P output=../../outputs/mlflow/baseline


In [ ]:
%%bash
set -e
mlflow ui --backend-store-uri mlruns


Use the MLflow UI to compare runs and reproduce experiments via logged parameters and artifacts.
